<a href="https://colab.research.google.com/github/ValiantChiu/Computer_Vision_AICSP/blob/main/VGG%E6%A8%A1%E5%9E%8B%E9%96%8B%E7%99%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.datasets.cifar10 import load_data
(trainX, trainY), (testX, testY) = load_data()
print('Train: X=%s, y=%s' % (trainX.shape, trainY.shape))
print('Test: X=%s, y=%s' % (testX.shape, testY.shape))

for i in range(9):
	plt.subplot(330 + 1 + i)
	plt.imshow(trainX[i])
plt.show()

In [ ]:
import sys
from numpy import asarray
from numpy import unique
from numpy import argmax
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.datasets.cifar10 import load_data
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD

In [ ]:
load_dataset()[0]

In [ ]:
def load_dataset():
	(trainX, trainY), (testX, testY) = load_data()
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)  #to_categorical()是一種One-Hot Encode
	return trainX, trainY, testX, testY

def prep_pixels(train, test):
	train_norm = train.astype('float32') #正常化處理
	test_norm = test.astype('float32')
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	return train_norm, test_norm

def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model


def define_model1():
	model = Sequential() #VGG 模型是一種CNN，先建立基本一block
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax')) #最後輸出模型需要分10類，所以輸出層設定10，使用softmax啟動函數
	# compile model
	opt = SGD(lr=0.001, momentum=0.9) #使用 0.001 學習率和 0.9 動量，都是很好訓練模型的開始
    #該模型優化設定為分類常用的SGD，Multi-Class Cross-Entropy Loss 常見的選擇
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model



def summarize_diagnostics(history):
	# plot loss
	plt.subplot(211)
	plt.title('Cross Entropy Loss')
	plt.plot(history.history['loss'], color='blue', label='train')
	plt.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	plt.subplot(212)
	plt.title('Classification Accuracy')
	plt.plot(history.history['accuracy'], color='blue', label='train')
	plt.plot(history.history['val_accuracy'], color='orange', label='test')
    # save plot to file
	filename = sys.argv[0].split('/')[-1]
	plt.savefig(filename + '_plot.png')
	plt.close()
def run_test_main(trainX, trainY, testX, testY):
	model = define_model()
	model.summary()
	history = model.fit(trainX, trainY, epochs=100, batch_size=64, validation_data=(testX, testY), verbose=0)
	summarize_diagnostics(history)
	loss, acc = model.evaluate(testX, testY,verbose=2)
 	#summarize_diagnostics(history)
	print(acc)
	model.save('final_model.h5')

In [ ]:
trainX, trainY, testX, testY = load_dataset()
trainX, testX = prep_pixels(trainX, testX)

In [ ]:
run_test_main(trainX, trainY, testX, testY)